Packages:

- [JuMP](https://github.com/JuliaOpt/JuMP.jl) Modeling framework, works with nonconvex problems
- [Ipopt](https://github.com/JuliaOpt/Ipopt.jl) Interior point optimizer

In [2]:
using JuMP
using Ipopt

# Same example using JuMP

Is there a better way to pass parameters? How to store intermediate expressions? Support for intermediate complex expressions?

In [3]:
function chsh_jump(x1,x2,x3,x4)
# variable x(1:2) Alice angles, x(3:4) Bob angles
    c = 0;
    for s = 1:2
        if s == 1
            alpha = x1;
        else
            alpha = x2;
        end
        for a = 1:2
           if a == 1
                A = [cos(alpha)
                     sin(alpha)];
            else
                A = [-sin(alpha)
                     cos(alpha)];
            end
            for t = 1:2
                if t == 1
                    beta = x3;
                else
                    beta = x4;
                end
                for b = 1:2
                    if b == 1
                        B = [cos(beta)
                             sin(beta)];
                    else
                        B = [-sin(beta)
                             cos(beta)];
                    end
                    ov = kron(A, B)' * [0; 1; -1; 0]/sqrt(2);
                    c = c + (-1)^((a-1)+(b-1)+(s-1)*(t-1))*ov*ov;
                end
            end
        end
    end
    return c
end

chsh_jump (generic function with 1 method)

In [4]:
using JuMP
using Ipopt
model = Model(Ipopt.Optimizer)
JuMP.register(model, :chsh_jump, 4, chsh_jump, autodiff=true)
x0 = rand(4)*2*pi
@variable(model, x1, start = x0[1])
@variable(model, x2, start = x0[2])
@variable(model, x3, start = x0[3])
@variable(model, x4, start = x0[4])
@NLobjective(model, Max, chsh_jump(x1, x2, x3, x4))
JuMP.optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ